## Assignment 3: Viterbi Algorithm

**Student:** Guillem Amat (ga98)

<br>

### Importing Packages

In [246]:
from typing import List, Tuple, Dict
import pandas as pd
import numpy as np
import pdb
import nltk
import os

In [248]:
os.chdir(r'C:\Users\guill\Desktop\Current Semester\Natural Language Processing\Homeworks\Homework_5')

<br>

### Importing the data

In [278]:
# Download packages if not available
nltk.download('universal_tagset')
nltk.download('brown')

In [250]:
positional_tagging = nltk.corpus.brown.tagged_sents(tagset='universal')[:10000]

In [251]:
positional_tagging

[[('The', 'DET'), ('Fulton', 'NOUN'), ('County', 'NOUN'), ('Grand', 'ADJ'), ('Jury', 'NOUN'), ('said', 'VERB'), ('Friday', 'NOUN'), ('an', 'DET'), ('investigation', 'NOUN'), ('of', 'ADP'), ("Atlanta's", 'NOUN'), ('recent', 'ADJ'), ('primary', 'NOUN'), ('election', 'NOUN'), ('produced', 'VERB'), ('``', '.'), ('no', 'DET'), ('evidence', 'NOUN'), ("''", '.'), ('that', 'ADP'), ('any', 'DET'), ('irregularities', 'NOUN'), ('took', 'VERB'), ('place', 'NOUN'), ('.', '.')], [('The', 'DET'), ('jury', 'NOUN'), ('further', 'ADV'), ('said', 'VERB'), ('in', 'ADP'), ('term-end', 'NOUN'), ('presentments', 'NOUN'), ('that', 'ADP'), ('the', 'DET'), ('City', 'NOUN'), ('Executive', 'ADJ'), ('Committee', 'NOUN'), (',', '.'), ('which', 'DET'), ('had', 'VERB'), ('over-all', 'ADJ'), ('charge', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('election', 'NOUN'), (',', '.'), ('``', '.'), ('deserves', 'VERB'), ('the', 'DET'), ('praise', 'NOUN'), ('and', 'CONJ'), ('thanks', 'NOUN'), ('of', 'ADP'), ('the', 'DET'), ('City

<br>

### Q1: Creating Matrixes

**Creating Initial tag-tag and tag-word lists**

In [252]:
# Creating a (tag_{i}, tag_{i+1}) list of tuples for every sentence in the positional_tagging corpus
tag_tag = [(tag[1], sentence[i+1][1]) for sentence in positional_tagging
           for i, tag in enumerate(sentence[:-1])]

In [253]:
# Creating a (tag_{i}, word_{i}) list of tuples for every sentence in the positional_tagging corpus: Extracting tuples
word_tag = [word for sentence in positional_tagging for word in sentence]

<br>

**Transition Matrix**

In [254]:
# Setting up components to create Transition Matrix
tag_vocabulary = set([tag[0] for tag in tag_tag])
tag_dictionary = {tag: i for i, tag in enumerate(list(tag_vocabulary))}
transition_matrix = np.zeros((len(tag_vocabulary), len(tag_vocabulary)))

In [255]:
# Populating the transition_matrix
for tag in tag_tag:
    tag_i  = tag_dictionary[tag[0]]
    tag_i1 = tag_dictionary[tag[1]]
    transition_matrix[tag_i, tag_i1] += 1

In [256]:
# Smoothing & converthing tag counts to probabilities
alpha = 0.01
V_tag  = len(tag_vocabulary)
d_tag  = (np.sum(transition_matrix, axis=1) + alpha*V_tag)[:, None] # We need to use [:, None] to make column division explicit
transition_matrix = (transition_matrix+alpha)/d_tag

In [257]:
# Checking Transition Matrix
df_tm = pd.DataFrame(transition_matrix, columns = tag_dictionary, index = tag_dictionary); df_tm.round(2)

,CONJ,PRT,X,NUM,ADJ,PRON,ADP,NOUN,.,ADV,VERB,DET
CONJ,0.00,0.02,0.00,0.02,0.12,0.05,0.07,0.29,0.02,0.09,0.16,0.16
PRT,0.01,0.01,0.00,0.01,0.02,0.00,0.09,0.04,0.05,0.03,0.66,0.08
X,0.02,0.00,0.46,0.00,0.00,0.00,0.07,0.11,0.26,0.01,0.05,0.00
NUM,0.03,0.01,0.00,0.02,0.07,0.01,0.14,0.38,0.25,0.03,0.04,0.01
ADJ,0.03,0.02,0.00,0.01,0.06,0.00,0.08,0.67,0.09,0.01,0.02,0.01
PRON,0.01,0.02,0.00,0.00,0.01,0.01,0.05,0.01,0.08,0.06,0.73,0.01
ADP,0.00,0.01,0.00,0.04,0.08,0.05,0.02,0.29,0.01,0.01,0.04,0.44
NOUN,0.05,0.02,0.00,0.01,0.02,0.02,0.23,0.21,0.26,0.02,0.14,0.01
.,0.09,0.02,0.00,0.02,0.05,0.06,0.11,0.18,0.14,0.06,0.13,0.12
ADV,0.01,0.03,0.00,0.02,0.15,0.04,0.14,0.04,0.14,0.10,0.26,0.08


<br>

**Observation Matrix**

In [258]:
# Setting up components to create Observation Matrix
word_vocabulary    = set([word[0] for word in word_tag])
word_dictionary    = {word: i for i, word in enumerate(list(word_vocabulary))}
observation_matrix = np.zeros((len(tag_vocabulary), len(word_vocabulary))) 

In [259]:
for wt in word_tag:
    tag  = tag_dictionary[wt[1]] # Recycling this dictionary from the Transition Matrix
    word = word_dictionary[wt[0]]
    observation_matrix[tag, word] += 1

In [260]:
# Smoothing & converting word-tag counts to probabilities
alpha = 0.01
V_word   = len(word_vocabulary)
d_word   = (np.sum(observation_matrix, axis=1) + alpha*V_word)[:, None] # We need to use [:, None] to make column division explicit
o_m = (observation_matrix + alpha)/d_word 

<br>

OOV Methods for Observation Matrix

In [ ]:
#Alternative approach by setting all tags to have equal probability
# observation_matrix = np.append(o_m, np.ones(len(tag_vocabulary),).reshape(-1,1), axis = 1)
#word_dictionary['OOV'] = observation_matrix.shape[1]-1

In [261]:
# Adding OOV observation to matrix
observation_matrix = np.append(o_m, d_word/np.sum(d_word), axis=1) #np.append(o_m, np.ones(len(tag_vocabulary),).reshape(-1,1), axis = 1)
word_dictionary['OOV'] = observation_matrix.shape[1]-1

<br>

In [262]:
# Checking Transition Matrix
df_om = pd.DataFrame(observation_matrix, columns = word_dictionary, index = tag_dictionary)
df_om.iloc[:5, -5:]

,explained,black-bearded,poised,undertaken,OOV
CONJ,1.459240e-06,0.000001,1.459240e-06,1.459240e-06,0.030787
PRT,1.884702e-06,0.000002,1.884702e-06,1.884702e-06,0.023837
X,2.045492e-05,0.000020,2.045492e-05,2.045492e-05,0.002196
NUM,2.656833e-06,0.000003,2.656833e-06,2.656833e-06,0.016910
ADJ,6.002444e-07,0.000061,6.002444e-07,6.002444e-07,0.074846


<br>

**Initial State Distribution**

Not happy with this... very convoluted, need to improve.

In [263]:
# Creating a dictionary to 
first_tag = [sentence[0][1] for sentence in positional_tagging]

In [264]:
# Counting the number of times each positional tag appears in the corpus
initial_state = {}

for tag in first_tag:
    initial_state[tag] = initial_state.get(tag, 0) + 1 

In [265]:
# Dividing the total amount of counts times the total count
count = sum(initial_state.values())

for k, v in initial_state.items():
    initial_state[k] = v/count

In [266]:
# Creating an array that will hold our dictionary values in the order of the transition matrix
tag_list = [None] * len(tag_dictionary.items())

for k, v in initial_state.items():
    tag_list[tag_dictionary[k]] = v
    pass
    
tag_array = np.array(tag_list).reshape(1, -1)

<br>

### Q2: Implementing Viterbi Algorithm

In [267]:
def viterbi(observations: List[str], pi: np.array, A: np.array, B: np.array) -> List[Tuple[str, str]]:
    
    #Setting up an initial matrix to hold our values and a list with the result
    final_tag_list = []
    viterbi_matrix = np.zeros((A.shape[0], len(observations))) 
    
    #Calculating initial Viterbi matrix column
    viterbi_matrix[:, 0] = np.log(pi * B[:, word_dictionary[observations[0]]])
    
    #Finding max value in the initial Viterbi column
    max_tag = np.argmax(viterbi_matrix[:, 0])
    final_tag_list.append(max_tag)
    
    #Iterating over every column
    for index in range(1, len(observations)):
        
        if observations[index] in word_dictionary.keys():
            viterbi_matrix[:, index] = np.log(A[max_tag, :] * B[:, word_dictionary[observations[index]]])
        else:
            viterbi_matrix[:, index] = np.log(A[max_tag, :] * B[:, word_dictionary['OOV']])
        
        #Finding max value in column iteration:= tag(i-1) for next iteration
        max_tag = np.argmax(viterbi_matrix[:, index])
        final_tag_list.append(max_tag)
        pass
    
    #Producing final results by looking at indexes of words
    inv = {val: key for key, val in tag_dictionary.items()}
    result = [inv[tag] for tag in final_tag_list]

    return list(zip(observations, result))  

<br>

### Q3: Testing Viterbi Algorithm

In [268]:
test = nltk.corpus.brown.tagged_sents(tagset='universal')[10150:10153]

In [269]:
results = [viterbi(extracted_sentence, tag_array, transition_matrix, observation_matrix)
          for extracted_sentence in 
          [[word[0] for word in sentence] for sentence in test]]

<br>

**Test Results**

In [270]:
list(zip(results[0], [tup[1] for tup in test[0]]))

[(('Those', 'DET'), 'DET'),
 (('coming', 'VERB'), 'VERB'),
 (('from', 'ADP'), 'ADP'),
 (('other', 'ADJ'), 'ADJ'),
 (('denominations', 'NOUN'), 'NOUN'),
 (('will', 'VERB'), 'VERB'),
 (('welcome', 'VERB'), 'VERB'),
 (('the', 'DET'), 'DET'),
 (('opportunity', 'NOUN'), 'NOUN'),
 (('to', 'ADP'), 'PRT'),
 (('become', 'VERB'), 'VERB'),
 (('informed', 'VERB'), 'VERB'),
 (('.', '.'), '.')]

In [271]:
list(zip(results[1], [tup[1] for tup in test[1]]))

[(('The', 'DET'), 'DET'),
 (('preparatory', 'NOUN'), 'ADJ'),
 (('class', 'NOUN'), 'NOUN'),
 (('is', 'VERB'), 'VERB'),
 (('an', 'DET'), 'DET'),
 (('introductory', 'NOUN'), 'ADJ'),
 (('face-to-face', 'NOUN'), 'ADJ'),
 (('group', 'NOUN'), 'NOUN'),
 (('in', 'ADP'), 'ADP'),
 (('which', 'DET'), 'DET'),
 (('new', 'ADJ'), 'ADJ'),
 (('members', 'NOUN'), 'NOUN'),
 (('become', 'VERB'), 'VERB'),
 (('acquainted', 'VERB'), 'VERB'),
 (('with', 'ADP'), 'ADP'),
 (('one', 'NUM'), 'NUM'),
 (('another', 'DET'), 'DET'),
 (('.', '.'), '.')]

In [272]:
list(zip(results[2], [tup[1] for tup in test[2]]))

[(('It', 'PRON'), 'PRON'),
 (('provides', 'VERB'), 'VERB'),
 (('a', 'DET'), 'DET'),
 (('natural', 'ADJ'), 'ADJ'),
 (('transition', 'NOUN'), 'NOUN'),
 (('into', 'ADP'), 'ADP'),
 (('the', 'DET'), 'DET'),
 (('life', 'NOUN'), 'NOUN'),
 (('of', 'ADP'), 'ADP'),
 (('the', 'DET'), 'DET'),
 (('local', 'ADJ'), 'ADJ'),
 (('church', 'NOUN'), 'NOUN'),
 (('and', 'CONJ'), 'CONJ'),
 (('its', 'DET'), 'DET'),
 (('organizations', 'NOUN'), 'NOUN'),
 (('.', '.'), '.')]

<br>

**Accuracy Computation**

In [273]:
class style:
    start = '\033[1m'
    end = '\033[0m'

In [275]:
res = list(zip([tup[1] for sentence in results for tup in sentence], [tup[1] for sentence in test for tup in sentence]))

In [276]:
accuracy = np.sum([tup[0] == tup[1] for tup in res])/len(res)* 100

In [277]:
print(style.start + 'Accuracy: ' + style.end + f'{accuracy.round(2)}%')

Accuracy: 91.49%
